In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
from barbar import Bar

In [2]:
from sklearn.metrics import balanced_accuracy_score

In [3]:
from torch.utils.data import Dataset, DataLoader

In [4]:
import pandas as pd
import pickle

In [5]:
import torch.utils.data as utils

In [6]:
import numpy as np

In [7]:
from torch.optim import lr_scheduler

In [8]:
DATAPATH = 'data/features/'
MODELPATH = 'output/models/'

### STEP 2: LOADING DATASET 

In [9]:
class CattleSoundDataset(Dataset):
    """ FreeSound dataset."""

    # Initialize your data, download, etc.
    def __init__(self, X, y):
        
        self.len = X.shape[0]
        self.x_data = torch.from_numpy(X)
        self.y_data = torch.from_numpy(y)

    def __getitem__(self, index):
        return (self.x_data[index], self.y_data[index])

    def __len__(self):
        return self.len

In [10]:
train = np.load(DATAPATH+'X_mfcc_001.npy')

In [11]:
labels = np.load(DATAPATH+'y.npy')

In [12]:
print('train:', train.shape)
print('labels:', labels.shape)


train: (1532, 60, 35)
labels: (1532,)


In [13]:
train_dataset = CattleSoundDataset(train, labels)

### STEP 2: MAKING DATASET ITERABLE

In [14]:
batch_size = 32

In [15]:
transformations = transforms.Compose([transforms.ToTensor()])

In [16]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size= batch_size, 
                                           shuffle=True)

### STEP 3: CREATE MODEL CLASS

In [17]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=(1,1)),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=(1,1)),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

        self.max_pool = nn.MaxPool2d(2)
        self._init_weights()
        
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.zeros_(m.bias)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.max_pool(x)
        return x

In [18]:
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        
        self.conv = nn.Sequential(
            ConvBlock(in_channels=1, out_channels=32),
            ConvBlock(in_channels=32, out_channels=64),
        )
        
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(7680, 1024),
            nn.PReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.1),
            nn.Linear(1024, num_classes),
        )

    def forward(self, x):
        out = self.conv(x)
        out = out.view(out.size(0), -1)
        #x = torch.mean(x, dim=3)
        #x, _ = torch.max(x, dim=2)
        out = self.fc(out)
        return out

### STEP 4: LOAD MODEL PRETRAINED 

In [19]:
model = CNNModel(num_classes=3, )

In [21]:
state = torch.load(MODELPATH+'cnn_pretrained.model')

In [22]:
model.load_state_dict(state['state_dict'])

In [23]:
#######################
#  USE GPU FOR MODEL  #
#######################

if torch.cuda.is_available():
    model.cuda()

### STEP 5: INSTANTIATE LOSS CLASS

In [24]:
criterion = nn.CrossEntropyLoss().cuda()

### STEP 6: LOAD OPTIMIZER 

In [25]:
learning_rate = 0.001

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [26]:
optimizer.load_state_dict(state['optimizer'])

In [27]:
num_epochs = state['epoch']

### STEP 7: TRAIN THE MODEL

In [28]:
niter = 0
for epoch in range(num_epochs):
    print('Epoch: {}'.format(epoch+1))
    
    for i, (images, labels) in enumerate(Bar(train_loader)):
        

        #######################
        #  USE GPU FOR MODEL  #
        #######################
        if torch.cuda.is_available():
            images = Variable(images.unsqueeze(1).cuda())
            labels = Variable(labels.cuda())
        else:
            images = Variable(images.unsqueeze(1))
            labels = Variable(labels)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        #images = images.unsqueeze(1).type(torch.FloatTensor).cuda()
        outputs = model(images.to(dtype=torch.float))
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()

Epoch: 1
1532/1532: [===============================>] - ETA 0.0s
Epoch: 2
1532/1532: [===============================>] - ETA 0.0s
Epoch: 3
1532/1532: [===============================>] - ETA 0.0s
Epoch: 4
1532/1532: [===============================>] - ETA 0.0s
Epoch: 5
1532/1532: [===============================>] - ETA 0.0s
Epoch: 6
1532/1532: [===============================>] - ETA 0.0s
Epoch: 7
1532/1532: [===============================>] - ETA 0.0s
Epoch: 8
1532/1532: [===============================>] - ETA 0.0s
Epoch: 9
1532/1532: [===============================>] - ETA 0.0s
Epoch: 10
1532/1532: [===============================>] - ETA 0.0s
Epoch: 11
1532/1532: [===============================>] - ETA 0.0s
Epoch: 12
1532/1532: [===============================>] - ETA 0.0s
Epoch: 13
1532/1532: [===============================>] - ETA 0.0s
Epoch: 14
1532/1532: [===============================>] - ETA 0.0s
Epoch: 15
1532/1532: [===============================>] - ETA 0.0s
Epoc

### STEP 8: SAVING THE MODEL

In [30]:
state = {
    'epoch': num_epochs,
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict()
}
torch.save(state, MODELPATH+'cnn_004.model')

In [31]:
model.state_dict()

OrderedDict([('conv.0.conv1.0.weight',
              tensor([[[[-3.3157e-01, -7.9121e-01, -5.9977e-01],
                        [ 7.4682e-01, -9.2104e-01,  3.3565e-01],
                        [ 4.2988e-01, -2.5153e-01,  7.7717e-02]]],
              
              
                      [[[-7.5641e-01, -4.9752e-01,  2.2808e-01],
                        [-1.7059e-01, -2.1400e-01,  3.7634e-01],
                        [ 4.2050e-01,  3.6550e-01, -5.2610e-02]]],
              
              
                      [[[ 1.1714e-01,  8.5059e-01,  5.1274e-01],
                        [-6.4502e-01, -1.8165e-01, -2.4872e-01],
                        [-3.3258e-01,  7.1543e-02, -1.4115e-01]]],
              
              
                      [[[-8.5607e-01,  2.1676e-01, -2.3662e-01],
                        [-2.6473e-01, -2.2716e-02, -7.7658e-01],
                        [-3.3861e-01, -1.7078e-01, -6.6343e-02]]],
              
              
                      [[[-7.1408e-01,  1.0734e+00, -6